In [9]:
import dash
import dash_bootstrap_components as dbc
import dash_core_components as dcc
import dash_html_components as html
from dash.dependencies import Input, Output
from flask_babel import get_locale, gettext




import pandas as pd
import plotly.express as px
import plotly.graph_objs as go
import numpy as np


In [12]:
import dash
import dash_bootstrap_components as dbc
import dash_core_components as dcc
import dash_html_components as html
from dash.dependencies import Input, Output
from flask_babel import get_locale, gettext



import pandas as pd
import plotly.express as px
import plotly.graph_objs as go

df_fb = pd.read_csv('../static/csv/facebook/movement_countries.csv')
countries = sorted(df_fb.end_name.unique())
df_fb.rename(columns = {'ds':'date'}, inplace = True)




df_fb = df_fb[df_fb['end_name'].isin(['austria','belgium' ,'croatia' ,'czech republic' ,'denmark',
 'estonia' ,'finland' ,'france' ,'germany', 'greece', 'iceland', 'ireland',
 'italy','lithuania' ,'luxembourg', 'netherlands', 'norway',
 'romania', 'serbia', 'spain', 'sweden', 'switzerland',
 'united kingdom', 'portugal', 'slovenia',  'hungary', 'poland',
 'bosnia and herzegovina', 'slovakia', 'bulgaria'])]

df_fb = df_fb[df_fb['start_name'].isin(['austria','belgium' ,'croatia' ,'czech republic' ,'denmark',
 'estonia' ,'finland' ,'france' ,'germany', 'greece', 'iceland', 'ireland',
 'italy','lithuania' ,'luxembourg', 'netherlands', 'norway',
 'romania', 'serbia', 'spain', 'sweden', 'switzerland',
 'united kingdom', 'portugal', 'slovenia',  'hungary', 'poland',
 'bosnia and herzegovina', 'slovakia', 'bulgaria' ])]


baseline_start = '2020-09-01'
baseline_end = '2020-09-07'



df_in = df_fb.groupby(['date', 'end_name']).agg({'travel_counts': 'sum'}).reset_index()
df_in = df_in[df_in.date <= baseline_end]
df_in = df_in[df_in.date >= baseline_start]
df_baseline_in = df_in.groupby(['end_name']).agg({'travel_counts': 'mean'}).reset_index()
df_baseline_in.rename(columns = {'travel_counts':'travel_counts_baseline'}, inplace = True)


df_out = df_fb.groupby(['date', 'start_name']).agg({'travel_counts': 'sum'}).reset_index()
df_out = df_out[df_out.date <= baseline_end]
df_out = df_out[df_out.date >= baseline_start]
df_baseline_out = df_out.groupby(['start_name']).agg({'travel_counts': 'mean'}).reset_index()
df_baseline_out.rename(columns = {'travel_counts':'travel_counts_baseline'}, inplace = True)


df_owid = pd.read_csv('../static/csv/owid.csv')
df_owid['location'] = df_owid['location'].str.lower()

countries = sorted(df_owid.location.unique())

df_owid = df_owid[['date','location','new_cases_smoothed_per_million',
                   'continent','population',
                   'new_cases_smoothed','reproduction_rate','stringency_index']]

df_owid = df_owid[df_owid['location'].isin(['austria','belgium' ,'croatia' ,'czech republic' ,'denmark',
 'estonia' ,'finland' ,'france' ,'germany', 'greece', 'iceland', 'ireland',
 'italy','lithuania' ,'luxembourg', 'netherlands', 'norway',
 'romania', 'serbia', 'spain', 'sweden', 'switzerland',
 'united kingdom', 'portugal', 'slovenia',  'hungary', 'poland',
 'bosnia and herzegovina', 'slovakia', 'bulgaria' ])]



df_owid = df_owid[df_owid['date']>=baseline_start]
df_owid.to_csv("../static/csv/owid_wave2_eu.csv")


#print(df_owid[df_owid.location=='portugal'])
df = pd.merge(df_fb, df_owid, how='left', left_on=['date','start_name'], 
              right_on=['date','location']).drop(columns= ['location']) 
df.rename(columns = {'new_cases_smoothed_per_million':'start_new_cases_smoothed_per_million',
                     'reproduction_rate':'start_reproduction_rate',
                    'population':'start_population','new_cases_smoothed':'start_new_cases_smoothed'}, inplace = True)




df = pd.merge(df, df_owid, how='left', 
              left_on=['date','end_name','continent'], 
              right_on=['date','location','continent']).drop(columns= ['location']) 

df.rename(columns = {'new_cases_smoothed_per_million':'end_new_cases_smoothed_per_million',
                     'population':'end_population',
                     'new_cases_smoothed':'end_new_cases_smoothed',
                     'reproduction_rate':'end_reproduction_rate'}, inplace = True)




df = df[df.date >= baseline_end]
df['end_reproduction_rate'] = df['end_reproduction_rate'].fillna(-10)
df['end_new_cases_smoothed'] = df['end_new_cases_smoothed'].fillna(-10)
df['end_new_cases_smoothed_per_million'] = df['end_new_cases_smoothed_per_million'].fillna(-10)
df['start_reproduction_rate'] = df['start_reproduction_rate'].fillna(-10)
df['start_new_cases_smoothed'] = df['start_new_cases_smoothed'].fillna(-10)
df['start_new_cases_smoothed_per_million'] = df['start_new_cases_smoothed_per_million'].fillna(-10)



df['covid_flow'] = df['start_new_cases_smoothed_per_million'] * df['travel_counts'] / 1000000
#df = df[df.continent =='Europe']



df_in = df.groupby(['date', 'end_name', 'end_population', 'end_new_cases_smoothed',
                 'end_reproduction_rate','end_new_cases_smoothed_per_million']).agg(
    {'covid_flow': 'sum', 'travel_counts': 'sum'}).reset_index()
df_in = pd.merge(df_in,df_baseline_in,how='left',left_on=['end_name'],right_on=['end_name'])


df_tot_travel_in = df_in.groupby(['date']).agg({'travel_counts':'sum','end_new_cases_smoothed':'sum'}).reset_index()



df_out = df.groupby(['date', 'start_name', 'start_population', 'start_new_cases_smoothed',
                 'start_reproduction_rate','start_new_cases_smoothed_per_million']).agg(
    {'covid_flow': 'sum', 'travel_counts': 'sum'}).reset_index()
df_out = pd.merge(df_out,df_baseline_out,how='left',left_on=['start_name'],right_on=['start_name'])






df_tot_travel_out = df_out.groupby(['date']).agg({'travel_counts':'sum','start_new_cases_smoothed':'sum'}).reset_index()


countries = df_in.end_name.unique()



def plot_tot_travel_in():
    fig = px.line(x=df_tot_travel_in.date.values,y=df_tot_travel_in.travel_counts.rolling(7).mean())

    fig.update_layout(template="plotly_white", margin=dict(l=0, r=0, t=30, b=0))


    fig.update_layout(
        height=600,
        title="Number of FB users who moved (in) between two EU countries in a 24 hour period (avg 7 days)",
        xaxis_title="date",
        yaxis_title="Total Flow"
    )
    return fig


def plot_tot_travel_out():
    fig = px.line(x=df_tot_travel_out.date.values,y=df_tot_travel_out.travel_counts.rolling(7).mean())

    fig.update_layout(template="plotly_white", margin=dict(l=0, r=0, t=30, b=0))


    fig.update_layout(
        height=600,
        title="Number of FB users who moved (out) between two EU countries in a 24 hour period (avg 7 days)",
        xaxis_title="date",
        yaxis_title="Total Flow"
    )
    return fig
    

def plot_tot_cases():
    fig = px.line(x=df_tot_travel.date.values,y=df_tot_travel.end_new_cases_smoothed.rolling(7).mean())

    fig.update_layout(template="plotly_white", margin=dict(l=0, r=0, t=30, b=0))


    fig.update_layout(
        height=600,
        title="Daily Number of covid cases in EU",
        xaxis_title="date",
        yaxis_title=gettext(
            get_translation(fr="cases", en="cases"))
    )
    return fig


def plot_in_per_million():
    plots = []
    for c in countries:
        dfc = df_in[df_in.end_name == c]
        deviation_travel = dfc['travel_counts'].rolling(7).mean()/dfc['travel_counts_baseline'].rolling(7).mean()
        cases_per_millions = dfc['end_new_cases_smoothed_per_million'].rolling(7).mean()
        
        plot = go.Scatter(x=dfc.date,y=deviation_travel, name=c)
        #plot = go.Scatter(x=deviation_travel, y=cases_per_millions, name=c,mode='markers')
        plots.append(plot)
    fig = go.Figure(data=plots, layout=go.Layout(barmode='group'))

    fig.update_layout(template="plotly_white", margin=dict(l=0, r=0, t=30, b=0))

    fig.update_layout(
        height=600,
        title="In Analysis",
        xaxis_title="date",
        yaxis_title="inflow contraction"
    )
    return fig


def plot_out_per_million():
    plots = []
    for c in countries:
        dfc = df_out[df_out.start_name == c]
        deviation_travel = dfc['travel_counts'].rolling(7).mean()/dfc['travel_counts_baseline'].rolling(7).mean()
        cases_per_millions = dfc['start_new_cases_smoothed_per_million'].rolling(7).mean()
        plot = go.Scatter(x=dfc.date,y=deviation_travel, name=c)
        #plot = go.Scatter(x=deviation_travel, y=cases_per_millions, name=c,mode='markers')
        plots.append(plot)
    fig = go.Figure(data=plots, layout=go.Layout(barmode='group'))

    fig.update_layout(template="plotly_white", margin=dict(l=0, r=0, t=30, b=0))

    fig.update_layout(
        height=600,
        title="Out Analysis",
        xaxis_title="date",
        yaxis_title="outflow contraction"
    )
    return fig


def plot_in_out():
    plots = []
    
    plots.append(go.Scatter(x=[0,1], y=[0,1], name="line"))
    for c in countries:
        dfc_out = df_out[df_out.start_name == c]
        dfc_in = df_in[df_in.end_name == c]
        

        maxv = np.nanmax(dfc_in.travel_counts.rolling(7).mean().values)
        print(maxv)

        plot = go.Scatter(x=dfc_in.travel_counts.rolling(7).mean()/maxv, 
                          y=dfc_out.travel_counts.rolling(7).mean()/maxv, 
                          name=c,mode='markers')
        plots.append(plot)

    
    fig = go.Figure(data=plots, layout=go.Layout(barmode='group'))

    fig.update_layout(template="plotly_white", margin=dict(l=0, r=0, t=30, b=0))
    #fig.update_xaxes(type="log")
    #fig.update_yaxes(type="log")
    fig.update_layout(
        height=600,
        title="In Out Analysis",
        xaxis_title="total in flow",
        yaxis_title="total out flow"
    )
    return fig

plot_in_out().show()
plot_tot_travel_in().show()
plot_tot_travel_out().show()
#plot_tot_cases().show()

plot_in_per_million().show()
plot_out_per_million().show()



79759.14285714286
102447.28571428571
11320.0
13791.0
17979.571428571428
40548.28571428572
25493.0
3032.4285714285716
5736.428571428572
156191.2857142857
246130.42857142858
19662.714285714286
29236.85714285714
16949.714285714286
84989.14285714286
2054.714285714286
32420.714285714286
100884.85714285714
6757.428571428572
71411.28571428571
22700.0
31028.0
9985.285714285714
23569.0
16299.142857142857
47819.42857142857
21136.714285714286
73229.85714285714
72691.42857142857


In [8]:
print(df_in)

            date                end_name  end_population  \
0     2020-09-07                 austria       9006400.0   
1     2020-09-07                 belgium      11589616.0   
2     2020-09-07  bosnia and herzegovina       3280815.0   
3     2020-09-07                bulgaria       6948445.0   
4     2020-09-07                 croatia       4105268.0   
...          ...                     ...             ...   
2560  2020-12-05                slovenia       2078932.0   
2561  2020-12-05                   spain      46754783.0   
2562  2020-12-05                  sweden      10099270.0   
2563  2020-12-05             switzerland       8654618.0   
2564  2020-12-05          united kingdom      67886004.0   

      end_new_cases_smoothed  end_reproduction_rate  \
0                    303.286                   1.29   
1                    504.714                   1.19   
2                    242.286                   1.16   
3                    125.714                   1.00   
4   

In [14]:

def plot_covid_import_index():
    plots = []
    for c in countries:
        dfc = df_in[df_in.end_name == c]
        
        index = 100*dfc['covid_flow'].rolling(7).mean()/dfc['end_new_cases_smoothed'].rolling(7).mean()
        
 
        plot = go.Scatter(x=dfc.date,y=index, name=c)
        #plot = go.Scatter(x=deviation_travel, y=cases_per_millions, name=c,mode='markers')
        plots.append(plot)
    fig = go.Figure(data=plots, layout=go.Layout(barmode='group'))

    fig.update_layout(template="plotly_white", margin=dict(l=0, r=0, t=30, b=0))

    fig.update_layout(
        height=600,
        title="Covid Import Index",
        xaxis_title="date",
        yaxis_title="covid import index in %"
    )
    return fig



def plot_covid_import_index_country(country = 'luxembourg'):
    import plotly.graph_objects as go
    from plotly.subplots import make_subplots
    df_country = df[df.end_name == country]
    countries = sorted(df_country.start_name.unique())
    
    colors=['rgb(31, 119, 180)', 'rgb(255, 127, 14)',
                       'rgb(44, 160, 44)', 'rgb(214, 39, 40)',
                       'rgb(148, 103, 189)', 'rgb(140, 86, 75)',
                       'rgb(227, 119, 194)', 'rgb(127, 127, 127)',
                       'rgb(188, 189, 34)', 'rgb(23, 190, 207)']
    
    # Create figure with secondary y-axis
    fig = make_subplots(specs=[[{"secondary_y": True}]])    
    i = 0
    for c in countries:
        i += 1
        dfc = df_country[df_country.start_name == c]
        
        index = 100*dfc['covid_flow'].rolling(7).mean()/dfc['end_new_cases_smoothed'].rolling(7).mean()
        
        
        # Add traces
        fig.add_trace(
            go.Scatter(x=dfc.date,y=index, 
                       name=c,legendgroup=c,marker_color = colors[i]),
            secondary_y=False,
        )
        
        # Add traces
        fig.add_trace(
            go.Scatter(x=dfc.date,y=dfc['start_new_cases_smoothed_per_million'], 
                       name=c,legendgroup=c,marker_color = colors[i],showlegend=False,line={'dash': 'dash'}),
            secondary_y=True,
        )
 

    fig.update_layout(template="plotly_white", margin=dict(l=0, r=0, t=30, b=0))
    

    # Add figure title
    fig.update_layout(
        title_text="Double Y Axis Example"
    )

    # Set x-axis title
    fig.update_xaxes(title_text="date")

    # Set y-axes titles
    fig.update_yaxes(title_text="covid import index in %", secondary_y=False)
    fig.update_yaxes(title_text="cases per million in origin country", secondary_y=True)

    

    fig.update_layout(
        height=600,
        title="Covid Import Index, Destination="+country,
    )
    return fig

plot_covid_import_index().show()
plot_covid_import_index_country().show()